In [1]:
def set_seed(seed=42):
    import torch
    import random
    import numpy as np

    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42) 

In [2]:
import random
import torch
from sklearn.model_selection import train_test_split
from transformers import BertConfig, BertForMaskedLM
import numpy as np

In [3]:
amino_acids = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"  # Standard amino acids
vocab = {aa: i for i, aa in enumerate(amino_acids)}
vocab["[MASK]"] = len(vocab)  # Add special [MASK] token
vocab["[PAD]"] = len(vocab)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
import torch
import torch.nn as nn
from torch.fft import rfft, irfft
from einops import rearrange
from transformers.modeling_outputs import MaskedLMOutput

class DiscreteTransform(nn.Module):
    def __init__(self, mode="fft", dim=-1):
        super().__init__()
        self.mode = mode
        self.dim = dim

    def forward(self, x):
        if self.mode == "fft":
            #print(f"fft {x.shape}")
            return rfft(x, dim=self.dim)

    def inverse(self, x, original_size=None):
        if self.mode == "fft":
            return irfft(x, dim=self.dim, n=original_size)

class BertEmbeddings(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.word_embeddings = nn.Embedding(config.vocab_size, config.hidden_size, padding_idx=config.pad_token_id)
        self.position_embeddings = nn.Embedding(config.max_position_embeddings, config.hidden_size)
        self.token_type_embeddings = nn.Embedding(config.type_vocab_size, config.hidden_size)

        self.LayerNorm = nn.LayerNorm(config.hidden_size, eps=config.layer_norm_eps)
        self.dropout = nn.Dropout(config.hidden_dropout_prob)

    def forward(self, input_ids, token_type_ids=None):
        input_shape = input_ids.size()
        seq_length = input_shape[1]

        position_ids = torch.arange(seq_length, dtype=torch.long, device=input_ids.device)
        position_ids = position_ids.unsqueeze(0).expand(input_shape)

        if token_type_ids is None:
            token_type_ids = torch.zeros(input_shape, dtype=torch.long, device=input_ids.device)

        word_embeddings = self.word_embeddings(input_ids)
        position_embeddings = self.position_embeddings(position_ids)
        token_type_embeddings = self.token_type_embeddings(token_type_ids)

        embeddings = word_embeddings + position_embeddings + token_type_embeddings
        embeddings = self.LayerNorm(embeddings)
        embeddings = self.dropout(embeddings)
        return embeddings  

class StaticConv(nn.Module):
    def __init__(self, d_model, seq_len):
        super().__init__()
        self.positional_encoding = nn.Parameter(
            torch.zeros(seq_len, d_model), requires_grad=False  # Non-learnable zeros
        )

    def forward(self, batch_size):
        # Expand positional encodings to include the batch dimension
        batch_positional_encoding = self.positional_encoding.unsqueeze(0).expand(batch_size, -1, -1)
        return batch_positional_encoding


class Abs(nn.Module):
    def forward(self, x):
        return torch.abs(x)
    
class OrchidOperator(nn.Module):
    def __init__(self, d, seq_len, d_filter=64, l_conv1d=3, transform_mode="fft"):
        super().__init__()
        self.d_model = d
        self.seq_len = seq_len

        self.in_linear = nn.Linear(d, 3 * d)
        self.out_linear = nn.Linear(d, d)

        width = d * 3
        self.short_filter = nn.Conv1d(width , width , kernel_size = l_conv1d , groups =width , padding = l_conv1d // 2)        
       
        self.static_conv = StaticConv(d, seq_len)
        #self.static_conv = StaticConv(d, d_filter, seq_len)

        self.conditioning_nn = nn.Sequential(
            nn.Conv1d(d, d, kernel_size = l_conv1d, padding=l_conv1d // 2, groups=d),
            DiscreteTransform(mode=transform_mode, dim=-1),
            Abs(),
            nn.Conv1d(d, d, kernel_size = l_conv1d, padding=l_conv1d // 2, groups=d),
        )

        self.transform = DiscreteTransform(mode=transform_mode, dim=-1)

    def forward(self, x):
        batch, seq_len, dim = x.size()
        orig_x = x

        x = self.in_linear(x)
        x = nn.functional.gelu(x)
        #_, _, v = torch.split(x, dim, dim=-1)
        _, _, v = x.split(self.d_model, dim=-1)
        #print(v.shape)

        h_adapt_f = self.conditioning_nn(rearrange(v, "b l d -> b d l"))
        
        h_static = self.static_conv(batch).transpose(1, 2)

        x = rearrange(x, "b l d -> b d l")
        x = self.short_filter(x)[..., : self.seq_len]
        s1, s2, v = x.split(self.d_model, dim=1)

        y = v * s1
        y = self.adaptive_conv(y, h_static, h_adapt_f)
        y = y * s2
        y = rearrange(y, "b d l -> b l d")
        y = self.out_linear(y)
        y = nn.functional.gelu(y)

        return self.out_linear(y)

    def adaptive_conv(self, x, h_static, h_adapt_f):
        x_seq_len = x.shape[-1]
        h_static_f = self.transform(h_static)
        x_f = self.transform(x)
        
        #print(f"self seq {self.seq_len}, h_static_f {h_static_f.shape}, h_adapt_f {h_adapt_f.shape},  x_f { x_f.shape},  h_static {h_static.shape}, x {x.shape}")
        
        y = self.transform.inverse(x_f * (h_static_f[:,:,:h_adapt_f.shape[-1]] + h_adapt_f),original_size=x_seq_len)
        _ = x_f * (h_static_f[:,:,:h_adapt_f.shape[-1]] + h_adapt_f)
        #print(f"y {y.shape}, before inverse {_.shape}")
        #return y[..., : self.seq_len]
        return y[..., : x_seq_len]
        

class OrchidBertLayer(nn.Module):
    def __init__(self, config, seq_len, l_conv1d):
        super().__init__()
        self.seq_len = seq_len
        self.orchid = OrchidOperator(
            d=config.hidden_size,
            seq_len=seq_len,
            d_filter=64,  # Adjust as needed
            l_conv1d=l_conv1d,   # Adjust as needed
            transform_mode="fft"
        )
        self.intermediate = nn.Linear(config.hidden_size, config.intermediate_size)
        self.output = nn.Linear(config.intermediate_size, config.hidden_size)
        self.norm1 = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.norm2 = nn.LayerNorm(config.hidden_size, eps=1e-12)
        self.dropout1 = nn.Dropout(config.hidden_dropout_prob)
        self.dropout2 = nn.Dropout(config.hidden_dropout_prob)
    
    def forward(self, hidden_states):
        # Pass hidden_states and positional embeddings to Orchid layer
        orchid_output = self.orchid(hidden_states)
        hidden_states = hidden_states + self.dropout1(orchid_output)
        hidden_states = self.norm1(hidden_states)

        # Feedforward block
        intermediate_output = self.intermediate(hidden_states)
        intermediate_output = nn.functional.gelu(intermediate_output)
        layer_output = self.output(intermediate_output)
        hidden_states = hidden_states + self.dropout2(layer_output)
        hidden_states = self.norm2(hidden_states)

        return hidden_states

class OrchidBERT(nn.Module):
    def __init__(self, config, seq_len, l_conv1d):
        super().__init__()
        self.config = config
        self.embeddings = BertEmbeddings(config)
        self.encoder_layers = nn.ModuleList(
            [OrchidBertLayer(config, seq_len, l_conv1d) for _ in range(config.num_hidden_layers)]
        )
        self.cls_head = nn.Linear(config.hidden_size, config.vocab_size)  # For Masked LM

    def forward(self, input_ids, attention_mask=None):
        # Compute embeddings
        embeddings = self.embeddings(input_ids)
        #position_embeddings = self.embeddings.position_embeddings.weight.unsqueeze(0)
        hidden_states = embeddings

        # Apply attention mask to embeddings if needed
        if attention_mask is not None:
            attention_mask = attention_mask.unsqueeze(-1)  # Match embedding dimensions
            hidden_states = hidden_states * attention_mask
        
        
        # Pass through Orchid-enhanced Transformer layers
        for layer in self.encoder_layers:
            #hidden_states = layer(hidden_states, position_embeddings)
            hidden_states = layer(hidden_states)

        logits = self.cls_head(hidden_states)
        probs = nn.functional.softmax(logits, dim=-1)
        
        return MaskedLMOutput(
            loss=None,
            logits=probs,
            hidden_states=[hidden_states],  # Optionally return hidden states if needed
            attentions=None      # Optionally return attentions if needed
        )

In [5]:
import torch
import torch.nn as nn

class FineTunedOrchidModel(nn.Module):
    def __init__(self, pretrained_model, num_targets, task_type="classification"):
        super(FineTunedOrchidModel, self).__init__()
        self.pretrained_model = pretrained_model  # Orchid-BERT
        self.task_type = task_type
        self.output_dim = num_targets
        
        # Task-specific head
        if self.task_type == "classification":
            self.head = nn.Sequential(
                nn.Linear(pretrained_model.config.hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, self.output_dim),
                #nn.Softmax(dim=1)
            )
        elif self.task_type == "regression":
            self.head = nn.Sequential(
                
                nn.Conv1d(pretrained_model.config.hidden_size, 128, kernel_size=3, padding=1),
                nn.ReLU(),
                nn.AdaptiveMaxPool1d(1),  # Outputs a single vector for each sequence
                nn.Flatten(),
                nn.Linear(128, 128),
                
                #nn.Linear(pretrained_model.config.hidden_size, 128),
                nn.ReLU(),
                nn.Linear(128, self.output_dim)
            )

    def forward(self, input_ids, attention_mask=None):
        # Forward pass through Orchid-BERT
        outputs = self.pretrained_model(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs.hidden_states[-1]# CLS token embedding
        
        if self.task_type == "regression": 
            #pooled_output = pooled_output[:, 0, :]
            #pooled_output = pooled_output.mean(dim=1)
            pooled_output = pooled_output.transpose(1, 2)
        return self.head(pooled_output)


In [6]:
from torch.utils.data import DataLoader, Dataset
from transformers import AdamW
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tqdm import tqdm

class ProteinDataset(Dataset):
    def __init__(self, data, max_len, pad_token="[PAD]", task_type="classification"):
        self.data = data
        self.max_len = max_len
        self.pad_token = pad_token
        self.task_type = task_type

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        input_ids, targets = self.data[idx]
        
        #print(len(input_ids),len(targets))

        # Pad input_ids to max_len
        padding_length = self.max_len - len(input_ids)
        if padding_length > 0:
            input_ids = input_ids + [vocab[self.pad_token]] * padding_length
        
        attention_mask = [1] * (self.max_len - padding_length) + [0] * padding_length

        # Adjust targets for each task
        if self.task_type == "classification":
            targets = targets + [-100] * padding_length  # For sequence tasks (secondary structure)
        elif self.task_type == "regression":
            targets = torch.tensor(targets, dtype=torch.float)  # For scalar tasks (fluorescence)

        #print(len(input_ids),len(targets),len(attention_mask))
        return (
            torch.tensor(input_ids),
            torch.tensor(targets),
            torch.tensor(attention_mask)
        )

In [14]:
from scipy.stats import spearmanr
def train(model, train_loader, val_loader, save_path, epochs=20, patience=3, task_type="classification"):
    optimizer = AdamW(model.parameters(), lr=5e-4)
    
    # Loss function selection
    if task_type == "classification":
        criterion = nn.CrossEntropyLoss(ignore_index=-100)  # Secondary structure
    elif task_type == "regression":
        criterion = nn.MSELoss()  # Fluorescence

    train_metrics, val_metrics = [], []
    spearman_metrics = []  # For regression tasks
    best_val_loss = float("inf")
    early_stop_counter = 0

    for epoch in range(epochs):
        # Training
        model.train()
        total_loss, total_metric, total = 0, 0, 0
        for batch in tqdm(train_loader, desc=f"Training Epoch {epoch + 1}"):
            input_ids, targets, attention_mask = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )
            optimizer.zero_grad()
            outputs = model(input_ids, attention_mask=attention_mask)
            if task_type == "classification":
                logits = outputs
                #print(logits.shape, targets.shape)
                loss = criterion(logits.view(-1, logits.size(-1)), targets.view(-1))
                predictions = logits.argmax(dim=-1)
                total_metric += (predictions == targets).sum().item()
                total += (targets != -100).sum().item()
            elif task_type == "regression":
                predictions = outputs.squeeze()
                loss = criterion(predictions, targets)
                total_metric += ((predictions - targets) ** 2).sum().item()  # MSE calculation
                total += len(targets)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        train_metrics.append(total_metric / total)  # Accuracy for classification, MSE for regression

        # Validation
        model.eval()
        val_loss, total_metric, total = 0, 0, 0
        all_predictions, all_targets = [], []
        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Validation Epoch {epoch + 1}"):
                input_ids, targets, attention_mask = (
                    batch[0].to(device),
                    batch[1].to(device),
                    batch[2].to(device),
                )
                outputs = model(input_ids, attention_mask=attention_mask)
                if task_type == "classification":
                    logits = outputs
                    loss = criterion(logits.view(-1, logits.size(-1)), targets.view(-1))
                    predictions = logits.argmax(dim=-1)
                    total_metric += (predictions == targets).sum().item()
                    total += (targets != -100).sum().item()
                elif task_type == "regression":
                    predictions = outputs.squeeze()
                    loss = criterion(predictions, targets)
                    total_metric += ((predictions - targets) ** 2).sum().item()  # MSE calculation
                    total += len(targets)
                    all_predictions.extend(predictions.cpu().numpy())
                    all_targets.extend(targets.cpu().numpy())
                val_loss += loss.item()

        val_metrics.append(total_metric / total)  # Accuracy for classification, MSE for regression
        
        if task_type == "regression":
            # Compute Spearman correlation
            spearman_corr, _ = spearmanr(all_predictions, all_targets)
            spearman_metrics.append(spearman_corr)
            print(f"Spearman Correlation: {spearman_corr:.4f}")

        
        print(f"Epoch {epoch + 1}: Train Metric={train_metrics[-1]:.4f}, Val Metric={val_metrics[-1]:.4f}, Val Loss={val_loss:.4f}")

        # Save the best model
        if val_loss < best_val_loss:
            print(f"Saving model with Val Loss={val_loss:.4f}")
            best_val_loss = val_loss
            torch.save(model.state_dict(), save_path)
        
        # Early stopping
        early_stop_counter = early_stop_counter + 1 if val_loss > best_val_loss else 0
        if early_stop_counter >= patience:
            print("Early stopping...")
            break

    return train_metrics, val_metrics



In [8]:
def test(model, test_loader, task_type="classification"):
    model.eval()
    correct, total, total_metric = 0, 0, 0
    criterion = nn.CrossEntropyLoss(ignore_index=-100) if task_type == "classification" else nn.MSELoss()
    all_predictions, all_targets = [], []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc="Testing"):
            input_ids, targets, attention_mask = (
                batch[0].to(device),
                batch[1].to(device),
                batch[2].to(device),
            )
            outputs = model(input_ids, attention_mask=attention_mask)
            if task_type == "classification":
                logits = outputs
                predictions = logits.argmax(dim=-1)
                total_metric += (predictions == targets).sum().item()
                total += (targets != -100).sum().item()
            elif task_type == "regression":
                predictions = outputs.squeeze()
                total_metric += ((predictions - targets) ** 2).sum().item()  # Sum of squared errors
                total += len(targets)
                all_predictions.extend(predictions.cpu().numpy())
                all_targets.extend(targets.cpu().numpy())
                
    if task_type == "regression":
        # Compute Spearman correlation
        spearman_corr, _ = spearmanr(all_predictions, all_targets)
        #spearman_metrics.append(spearman_corr)
        print(f"Spearman Correlation: {spearman_corr:.4f}")

    if task_type == "classification":
        accuracy = total_metric / total
        print(f"Test Accuracy: {accuracy:.4f}")
        return accuracy
    elif task_type == "regression":
        mse = total_metric / total  # Mean Squared Error
        print(f"Test MSE: {mse:.4f}")
        return mse


In [9]:
import lmdb
import os 
import json
import pickle

base_path = os.getcwd()
#path = os.path.join(base_path, "fluorescence", "fluorescence_test.lmdb")
# Path to the LMDB folder (not the .mdb file)
# Path to the LMDB folder (not the .mdb file itself)

def load_fluorescence_data(path):
    lmdb_path = os.path.join(base_path,path)

    # Open the LMDB environment
    env = lmdb.open(lmdb_path, readonly=True)

    # List to store the extracted data
    data_list = []

    # Start a transaction to read data
    with env.begin() as txn:
        cursor = txn.cursor()
        for key, value in cursor:
            # Decode the key and value (adjust decoding as needed)
            decoded_key = key.decode("utf-8")
            decoded_value = pickle.loads(value)
            data_list.append((decoded_key, decoded_value))  # Store as (key, value) tuples

    print(f"Total entries: {len(data_list)}")
    #print(f"Sample entry: {data_list[0]}")
    return data_list

In [10]:
fluorescence_data = {}
_keys = ["train", "test", "val"]
fluorescence_paths = ["fluorescence/fluorescence_train.lmdb","fluorescence/fluorescence_test.lmdb","fluorescence/fluorescence_valid.lmdb"]
fluorescence_input_label_pairs = {}

for key, path in zip(_keys, fluorescence_paths):
    # Load the data for the given key (train, test, val)
    fluorescence_data[key] = load_fluorescence_data(path)
    
    # Initialize the input-label pairs list for the given key
    fluorescence_input_label_pairs[key] = []

    # Process each data entry
    for entry in fluorescence_data[key][:-1]:
        # Extract the primary sequence and labels (ss3)
        primary_sequence = list(entry[1]["primary"])
        labels = entry[1]["log_fluorescence"][0]

        # Convert primary sequence to token IDs using the vocabulary
        inp_ids = [vocab[tok] for tok in primary_sequence]

        # Append the input-label pair to the list
        fluorescence_input_label_pairs[key].append((inp_ids, labels))

Total entries: 21447
Total entries: 27218
Total entries: 5363


In [11]:
secondary_structure_data = {}
secondary_structure_input_label_pairs = {}
_keys = ["train", "test", "val"]
secondary_structure_paths = ["secondary_structure/secondary_structure_train.lmdb","secondary_structure/secondary_structure_cb513.lmdb","secondary_structure/secondary_structure_valid.lmdb"]

for key, path in zip(_keys, secondary_structure_paths):
    # Load the data for the given key (train, test, val)
    secondary_structure_data[key] = load_fluorescence_data(path)
    
    # Initialize the input-label pairs list for the given key
    secondary_structure_input_label_pairs[key] = []

    # Process each data entry
    for entry in secondary_structure_data[key][:-1]:
        # Extract the primary sequence and labels (ss3)
        primary_sequence = list(entry[1]["primary"])
        labels = entry[1]["ss3"].tolist()

        # Convert primary sequence to token IDs using the vocabulary
        inp_ids = [vocab[tok] for tok in primary_sequence]

        # Append the input-label pair to the list
        secondary_structure_input_label_pairs[key].append((inp_ids, labels))

Total entries: 8679
Total entries: 514
Total entries: 2171


In [12]:
def train_and_test(task_type, train_data, val_data, test_data, seq_len, vocab, model_class, pretrained_model_path, output_model_path, num_targets=1,skip=[]):
    """
    Train and test a model for a given task type.

    Parameters:
        task_type (str): Task type, either "regression" or "classification".
        train_data (list): Training dataset.
        val_data (list): Validation dataset.
        test_data (list): Test dataset.
        seq_len (int): Sequence length to filter data and configure the model.
        vocab (dict): Vocabulary mapping.
        model_class (torch.nn.Module): The model class to be used.
        pretrained_model_path (str): Path to the pretrained model file.
        output_model_path (str): Path to save the fine-tuned model.
        num_targets (int): Number of output targets for the task.

    Returns:
        float: Test accuracy or metric after testing the model.
    """
    import torch
    from torch.utils.data import DataLoader
    
    cut_len = 10000000000
    
    # Filter data based on sequence length
    train_data = [item for item in train_data if len(item[0]) <= seq_len][:cut_len]
    val_data = [item for item in val_data if len(item[0]) <= seq_len][:cut_len]
    test_data = [item for item in test_data if len(item[0]) <= seq_len][:cut_len]
    
    old_seq_len = seq_len
    seq_len = max([len(item[0]) for item in train_data+val_data+test_data])

    # Prepare DataLoaders
    batch_size = 32
    train_loader = DataLoader(ProteinDataset(train_data, seq_len,task_type=task_type), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(ProteinDataset(val_data, seq_len,task_type=task_type), batch_size=batch_size)
    test_loader = DataLoader(ProteinDataset(test_data, seq_len,task_type=task_type), batch_size=batch_size)

    # Configure device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    # Load pretrained model
    config = BertConfig(
        vocab_size=len(vocab),
        hidden_size=128,
        num_hidden_layers=3,
        num_attention_heads=2,
        intermediate_size=256,
        hidden_dropout_prob=0.1,
        max_position_embeddings=old_seq_len,
        output_hidden_states=True
    )

    if model_class == OrchidBERT:
        pretrained_model = model_class(config, seq_len=old_seq_len, l_conv1d=11)
    else:
        pretrained_model = model_class(config)
    pretrained_model.load_state_dict(torch.load(pretrained_model_path))
    pretrained_model.to(device)

    if output_model_path not in skip:
        # Initialize task-specific model
        model = FineTunedOrchidModel(pretrained_model, num_targets=num_targets, task_type=task_type)
        model.to(device)

        # Train the model
        print(f"Training model for task: {task_type}, sequence length: {seq_len}, pretrain model: {pretrained_model_path}")
        train(model, train_loader, val_loader, output_model_path, task_type=task_type)
    
    model = FineTunedOrchidModel(pretrained_model, num_targets=num_targets, task_type=task_type)
    model.load_state_dict(torch.load(output_model_path))
    model.to(device)
    
    # Test the model
    test_accuracy = test(model, test_loader, task_type=task_type)

    print(f"Test Accuracy for task: {task_type} at sequence length {seq_len} pretrain model {pretrained_model_path}: {test_accuracy:.4f}")

    return test_accuracy


In [16]:
# Iterate over different configurations and record metrics
seq_lengths = [1000, 2000]
tasks = ["regression","classification"]
pretrained_model_paths = ["transformer_model", "orchid_model_tmp"]
metrics = []

for seq_len in seq_lengths:
    for task in tasks:
        for pretrained_path in pretrained_model_paths:
            output_path = f"{pretrained_path}_{task}_{seq_len}_1"
            test_accuracy = train_and_test(
                task_type=task,
                train_data=fluorescence_input_label_pairs["train"] if task == "regression" else secondary_structure_input_label_pairs["train"],
                val_data=fluorescence_input_label_pairs["val"] if task == "regression" else secondary_structure_input_label_pairs["val"],
                test_data=fluorescence_input_label_pairs["test"] if task == "regression" else secondary_structure_input_label_pairs["test"],
                seq_len=seq_len,
                vocab=vocab,
                model_class=BertForMaskedLM if pretrained_path == "transformer_model" else OrchidBERT,
                pretrained_model_path=f"{pretrained_path}_{seq_len}",
                output_model_path=output_path,
                num_targets=1 if task == "regression" else 3,
                skip = ["transformer_model_classification_1000","orchid_model_tmp_classification_1000",
                        "transformer_model_classification_2000","orchid_model_tmp_classification_2000"]
            )
            metrics.append({
                "seq_len": seq_len,
                "task": task,
                "pretrained_model": pretrained_path,
                "test_accuracy": test_accuracy
            })

# Print all metrics
for metric in metrics:
    print(metric)

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_84\2904733188.py:58: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pretrained_model.load_state_dict(torch.load(pr

Training model for task: regression, sequence length: 237, pretrain model: transformer_model_1000


Training Epoch 1:   0%|                                                                        | 0/671 [00:00<?, ?it/s]C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_84\1911442212.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(targets),
Validation Epoch 1: 100%|████████████████████████████████████████████████████████████| 168/168 [00:03<00:00, 55.16it/s]


Spearman Correlation: 0.4734
Epoch 1: Train Metric=0.7197, Val Metric=0.6044, Val Loss=101.4733
Saving model with Val Loss=101.4733


Validation Epoch 2: 100%|████████████████████████████████████████████████████████████| 168/168 [00:03<00:00, 55.57it/s]


...

Epoch 13: Train Metric=0.7367, Val Metric=0.7175, Val Loss=45.0970


Validation Epoch 14: 100%|█████████████████████████████████████████████████████████████| 68/68 [00:14<00:00,  4.80it/s]


Epoch 14: Train Metric=0.7391, Val Metric=0.7163, Val Loss=46.1363
Early stopping...


Testing: 100%|█████████████████████████████████████████████████████████████████████████| 17/17 [00:03<00:00,  4.83it/s]


Test Accuracy: 0.6994
Test Accuracy for task: classification at sequence length 1632 pretrain model orchid_model_tmp_2000: 0.6994
{'seq_len': 1000, 'task': 'regression', 'pretrained_model': 'transformer_model', 'test_accuracy': 0.5384143093188811}
{'seq_len': 1000, 'task': 'regression', 'pretrained_model': 'orchid_model_tmp', 'test_accuracy': 0.2217750140031325}
{'seq_len': 1000, 'task': 'classification', 'pretrained_model': 'transformer_model', 'test_accuracy': 0.6203518980326961}
{'seq_len': 1000, 'task': 'classification', 'pretrained_model': 'orchid_model_tmp', 'test_accuracy': 0.6988085342200056}
{'seq_len': 2000, 'task': 'regression', 'pretrained_model': 'transformer_model', 'test_accuracy': 0.2282922454543786}
{'seq_len': 2000, 'task': 'regression', 'pretrained_model': 'orchid_model_tmp', 'test_accuracy': 0.36485722032385043}
{'seq_len': 2000, 'task': 'classification', 'pretrained_model': 'transformer_model', 'test_accuracy': 0.6134663341645885}
{'seq_len': 2000, 'task': 'classif